In [3]:
import pandas as pd
import sidetable
import regex as re
import numpy as np

In [4]:
df = pd.read_pickle("datos/attacks_pandas_8.pickle")

HIPOTESIS 

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

En este ejercicio nos queda por limpiar las columnas de age y species. Trabajaremos con el archivo generado en limpieza 1

In [5]:
df.sample()

,case_number,year,country,area,location,sex_,age,species_,siglo,injury,date,mes,fatal_(y/n)_limpio,sex,country_
7601,1816.09.03.R,<NA>,nan,NaN,NaN,NaN,NaN,Unspecified,NaN,FATAL,Reported 03-Sept-1816,Sept,Y,M,NaN


1 Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified".
- Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.
- Tendremos que crearnos una función que aplicaremos sobre nuestra columna species para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [6]:
patron_blanco = r".*[Ww](hite|HITE).*"
patron_tigre = r".*[Tt](iger|IGER).*"
patron_gris = r".*[Gg](rey|REY).*"
patron_limon = r".*[Ll](emon|EMON).*"
patron_toro = r".*[Bb](ull|ULL).*"

In [7]:
def clasificacion(col):
    col = str(col)
    if re.findall(patron_blanco, col):
        return "White Shark"
    elif re.findall(patron_tigre, col):
        return "Tiger Shark"
    elif re.findall(patron_gris, col):
        return "Grey Shark"
    elif re.findall(patron_limon, col):
        return "Lemon Shark"
    elif re.findall(patron_toro, col):
        return "Bull Shark"
    else:
        return "Unspecified"

In [8]:
df["species_"]= df["species_"].apply(clasificacion)

In [9]:
df.sample(2)

,case_number,year,country,area,location,sex_,age,species_,siglo,injury,date,mes,fatal_(y/n)_limpio,sex,country_
6182,1942.11.25,<NA>,nan,NaN,NaN,NaN,NaN,Unspecified,NaN,"Calf bitten, leg surgically amputated",25-Nov-1942,Nov,N,F,NaN
4435,1983.05.21,<NA>,nan,NaN,NaN,NaN,NaN,Unspecified,NaN,Right foot bitten,21-May-1983,May,N,M,NaN


Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
- Edad en formato string
- Edades separadas por &, or, to, >
- Edades con ?

- Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen.
- Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.
- Por último cambiad el tipo de la columna de string a integer.


In [10]:
#df["age"] = df["age"].str.replace("&", "")

In [11]:
patron_num = r"[0-99]+"

In [12]:
def num(col):
    if re.findall(patron_num, col):
        return int(patron_num)
    else: 
        col.split()

In [13]:
df["age"] = df["age"].apply(num)

ValueError: invalid literal for int() with base 10: '[0-99]+'

3 Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [ ]:
df["age"].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '60s',
       '51', '61', '59', '42', '6', '27', '23', '29', '39', '24', '12',
       '26', '71', '43', '9', '44', '14', '62', '52', '38', '68', '16',
       '47', '63', '70', '41', '40s', '53', '20s', '7', '66', '45', '74',
       '64', '8', '56', '49', '18 or 20', 'Teen', '30s', '77', '8 or 10',
       '84', '\xa0 ', ' ', '30 or 36', '6½', '5', ' 30', ' 28', "60's",
       '67', '>50', '? & 19', '21, 34,24 & 35', '30 & 32', '13 or 18',
       '7 or 8', '9 or 10', nan, 'young', '13 or 14'], dtype=object)